# Experimentos com AutoEncoder
## Detecção de anomalias em poços produtores de petróleo usando aprendizado de máquina

**_Principais Referências_**
  
- **A Realistic and Public Dataset with Rare Undesirable Real Events in Oil Wells** ([link](https://doi.org/10.1016/j.petrol.2019.106223)).
- **Github de referência do _benchmark_ proposto por Vargas (2019)** ([link](https://github.com/ricardovvargas/3w_dataset)).

## 1. Regras do benchmark proposto por Vargas (2019)

- Apenas instâncias reais com anomalias de tipos que têm períodos normais maiores ou iguais a vinte minutos foram utilizadas;

- Múltiplas rodadas de treinamento e validação realizadas, sendo o número de rodadas igual ao número de instâncias. Em cada rodada, amostras utilizadas para treinamento ou validação extraídas de apenas uma instância. Parte das amostras de normalidade utilizadas no treinamento (60%) e a outra parte para teste (40%). As amostras de anomalias somente devem ser utilizadas apenas teste, sendo, portanto, uma técnica de treinamento de classe única. O conjunto de teste deve ser composto pelo mesmo número de amostras de cada classe (normalidade e anormalidade);

- Em cada rodada, precisão, revogação e medida F1 devem ser computadas (valor médio e desvio padrão de cada métrica), sendo o valor médio da medida F1 considerado a principal métrica de desempenho.

## 1.1 Outras definições adotadas

- Uma estratégia de amostragem específica com janela deslizante foi usada para cada tipo de período. Em períodos normais, as primeiras observações são usadas para treinamento e as últimas são usadas para testes. Em períodos transientes, procura-se usar observações como um todo (apenas para teste). Em períodos de regime, as primeiras observações são privilegiadas (somente para teste);

- Antes de cada rodada de treinamento e teste:
    - As amostras utilizadas (não as instâncias) são adequadamente normalizadas com o z-score;
    - As variáveis de amostras (não as instâncias) usadas para treinamento que possuem um número de NaNs acima de um limite ou que têm um desvio padrão abaixo de um outro limite são descartadas.

- Todos os random_state necessários são atribuídos a uma constante para que os resultados sejam reproduzíveis.

## 2. Bibliotecas e configurações

In [1]:
# Artifício para alcular tempo total do notebook Jupyter
from datetime import datetime 
start_time = datetime.now()

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import logging
import warnings
import sys
sys.path.append('stac')
import nonparametric_tests as stac
from math import ceil
from matplotlib import pyplot as plt
from time import time
from pathlib import Path
from tsfresh.feature_extraction import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.dummy import DummyClassifier
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import ParameterGrid
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support

In [3]:
"""Using Auto Encoder with Outlier Detection"""
# Author: Yue Zhao <zhaoy@cmu.edu>
# License: BSD 2 clause

from __future__ import division
from __future__ import print_function

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.losses import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_array
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.multiclass import check_classification_targets


class AutoEncoder():
    """Auto Encoder (AE) is a type of neural networks for learning useful data
    representations unsupervisedly. Similar to PCA, AE could be used to
    detect outlying objects in the data by calculating the reconstruction
    errors. See :cite:`aggarwal2015outlier` Chapter 3 for details.

    Parameters
    ----------
    hidden_neurons : list, optional (default=[64, 32, 32, 64])
        The number of neurons per hidden layers.

    hidden_activation : str, optional (default='relu')
        Activation function to use for hidden layers.
        All hidden layers are forced to use the same type of activation.
        See https://keras.io/activations/

    output_activation : str, optional (default='sigmoid')
        Activation function to use for output layer.
        See https://keras.io/activations/

    loss : str or obj, optional (default=keras.losses.mean_squared_error)
        String (name of objective function) or objective function.
        See https://keras.io/losses/

    optimizer : str, optional (default='adam')
        String (name of optimizer) or optimizer instance.
        See https://keras.io/optimizers/

    epochs : int, optional (default=100)
        Number of epochs to train the model.

    batch_size : int, optional (default=32)
        Number of samples per gradient update.

    dropout_rate : float in (0., 1), optional (default=0.2)
        The dropout to be used across all layers.

    l2_regularizer : float in (0., 1), optional (default=0.1)
        The regularization strength of activity_regularizer
        applied on each layer. By default, l2 regularizer is used. See
        https://keras.io/regularizers/

    validation_size : float in (0., 1), optional (default=0.1)
        The percentage of data to be used for validation.

    preprocessing : bool, optional (default=False)
        If True, apply standardization on the data.

    verbose : int, optional (default=1)
        Verbosity mode.

        - 0 = silent
        - 1 = progress bar
        - 2 = one line per epoch.

        For verbosity >= 1, model summary may be printed.

    random_state : random_state: int, RandomState instance or None, optional
        (default=None)
        If int, random_state is the seed used by the random
        number generator; If RandomState instance, random_state is the random
        number generator; If None, the random number generator is the
        RandomState instance used by `np.random`.

    contamination : float in (0., 0.5), optional (default=0.1)
        The amount of contamination of the data set, i.e.
        the proportion of outliers in the data set. When fitting this is used
        to define the threshold on the decision function.

    Attributes
    ----------
    encoding_dim_ : int
        The number of neurons in the encoding layer.

    compression_rate_ : float
        The ratio between the original feature and
        the number of neurons in the encoding layer.

    model_ : Keras Object
        The underlying AutoEncoder in Keras.

    history_: Keras Object
        The AutoEncoder training history.

    decision_scores_ : numpy array of shape (n_samples,)
        The outlier scores of the training data.
        The higher, the more abnormal. Outliers tend to have higher
        scores. This value is available once the detector is
        fitted.

    threshold_ : float
        The threshold is based on ``contamination``. It is the
        ``n_samples * contamination`` most abnormal samples in
        ``decision_scores_``. The threshold is calculated for generating
        binary outlier labels.

    labels_ : int, either +1 or -1
        The binary labels of the training data. 0 stands for inliers
        and 1 for outliers/anomalies. It is generated by applying
        ``threshold_`` on ``decision_scores_``.
    """

    def __init__(self, hidden_neurons=[64, 32, 32, 64],
                 hidden_activation='relu', output_activation='sigmoid',
                 loss=mean_squared_error, optimizer='adam',
                 epochs=100, batch_size=32, dropout_rate=0.2,
                 l2_regularizer=0.1, validation_size=0.1, preprocessing=False,
                 verbose=1, random_state=None, contamination=0.1):
        self.contamination = contamination
        self.hidden_neurons = hidden_neurons
        self.hidden_activation = hidden_activation
        self.output_activation = output_activation
        self.loss = loss
        self.optimizer = optimizer
        self.epochs = epochs
        self.batch_size = batch_size
        self.dropout_rate = dropout_rate
        self.l2_regularizer = l2_regularizer
        self.validation_size = validation_size
        self.preprocessing = preprocessing
        self.verbose = verbose
        self.random_state = random_state


        # Verify the network design is valid
        if not self.hidden_neurons == self.hidden_neurons[::-1]:
            print(self.hidden_neurons)
            raise ValueError("Hidden units should be symmetric")

        self.hidden_neurons_ = self.hidden_neurons

    def _build_model(self):
        model = Sequential()
        # Input layer
        model.add(Dense(
            self.hidden_neurons_[0], activation=self.hidden_activation,
            input_shape=(self.n_features_,),
            activity_regularizer=l2(self.l2_regularizer)))
        model.add(Dropout(self.dropout_rate))

        # Additional layers
        for i, hidden_neurons in enumerate(self.hidden_neurons_, 1):
            model.add(Dense(
                hidden_neurons,
                activation=self.hidden_activation,
                activity_regularizer=l2(self.l2_regularizer)))
            model.add(Dropout(self.dropout_rate))

        # Output layers
        model.add(Dense(self.n_features_, activation=self.output_activation,
                        activity_regularizer=l2(self.l2_regularizer)))

        # Compile model
        model.compile(loss=self.loss, optimizer=self.optimizer)
        if self.verbose >= 1:
            print(model.summary())
        return model

    def fit(self, X, y=None):
        """Fit detector. y is ignored in unsupervised methods.

        Parameters
        ----------
        X : numpy array of shape (n_samples, n_features)
            The input samples.

        y : Ignored
            Not used, present for API consistency by convention.

        Returns
        -------
        self : object
            Fitted estimator.
        """
        # validate inputs X and y (optional)
        X = check_array(X)

        # Verify and construct the hidden units
        self.n_samples_, self.n_features_ = X.shape[0], X.shape[1]

        # Standardize data for better performance
        if self.preprocessing:
            self.scaler_ = StandardScaler()
            X_norm = self.scaler_.fit_transform(X)
        else:
            X_norm = np.copy(X)

        # Shuffle the data for validation as Keras do not shuffling for
        # Validation Split
        np.random.shuffle(X_norm)

        # Validate and complete the number of hidden neurons
        if np.min(self.hidden_neurons) > self.n_features_:
            raise ValueError("The number of neurons should not exceed "
                             "the number of features")
        self.hidden_neurons_.insert(0, self.n_features_)

        # Calculate the dimension of the encoding layer & compression rate
        self.encoding_dim_ = np.median(self.hidden_neurons)
        self.compression_rate_ = self.n_features_ // self.encoding_dim_

        # Build AE model & fit with X
        self.model_ = self._build_model()
        self.history_ = self.model_.fit(X_norm, X_norm,
                                        epochs=self.epochs,
                                        batch_size=self.batch_size,
                                        shuffle=True,
                                        validation_split=self.validation_size,
                                        verbose=self.verbose).history
        # Reverse the operation for consistency
        self.hidden_neurons_.pop(0)
        # Predict on X itself and calculate the reconstruction error as
        # the outlier scores. Noted X_norm was shuffled has to recreate
        if self.preprocessing:
            X_norm = self.scaler_.transform(X)
        else:
            X_norm = np.copy(X)

        pred_scores = self.model_.predict(X_norm)
        self.decision_scores_ = np.sqrt(np.sum(np.square(pred_scores - X_norm), axis=1)).ravel()
        
        self._process_decision_scores()
        return self

    def decision_function(self, X):
        """Predict raw anomaly score of X using the fitted detector.

        The anomaly score of an input sample is computed based on different
        detector algorithms. For consistency, outliers are assigned with
        larger anomaly scores.

        Parameters
        ----------
        X : numpy array of shape (n_samples, n_features)
            The training input samples. Sparse matrices are accepted only
            if they are supported by the base estimator.

        Returns
        -------
        anomaly_scores : numpy array of shape (n_samples,)
            The anomaly score of the input samples.
        """
        check_is_fitted(self, ['model_', 'history_'])
        X = check_array(X)

        if self.preprocessing:
            X_norm = self.scaler_.transform(X)
        else:
            X_norm = np.copy(X)

        # Predict on X and return the reconstruction errors
        pred_scores = self.model_.predict(X_norm)
        return np.sqrt(np.sum(np.square(pred_scores - X_norm), axis=1)).ravel()

    
    def _process_decision_scores(self, threshold=None):
        """Internal function to calculate key attributes:

        - threshold_: used to decide the binary label
        - labels_: binary labels of training data

        Returns
        -------
        self
        """

        self.threshold_ = np.percentile(self.decision_scores_,
                                     100 * (1 - self.contamination))
        
        self.labels_ = (self.decision_scores_ > self.threshold_).astype(
            'int').ravel()

        # calculate for predict_proba()
        self._mu = np.mean(self.decision_scores_)
        self._sigma = np.std(self.decision_scores_)

        return self
    
    def predict(self, X):
        """Predict if a particular sample is an outlier or not.

        Parameters
        ----------
        X : numpy array of shape (n_samples, n_features)
            The input samples.

        Returns
        -------
        is_inlier : ndarray of shape (n_samples,)
        Returns -1 for anomalies/outliers and +1 for inliers.
        """

        check_is_fitted(self, ['decision_scores_', 'threshold_', 'labels_'])
        
        is_inlier = np.ones(X.shape[0], dtype=int)
        is_inlier[self.decision_function(X) < self.threshold_] = -1        
        
        return is_inlier.ravel()

Using TensorFlow backend.


In [4]:
logging.getLogger('tsfresh').setLevel(logging.ERROR)
warnings.simplefilter(action='ignore')

In [5]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'

In [6]:
data_path = Path('./', 'data')
random_state = 1
events_names = {0: 'Normal',
                1: 'Aumento Abrupto de BSW',
                2: 'Fechamento Espúrio de DHSV',
                3: 'Intermitência Severa',
                4: 'Instabilidade de Fluxo',
                5: 'Perda Rápida de Produtividade',
                6: 'Restrição Rápida em CKP',
                7: 'Incrustação em CKP',
                8: 'Hidrato em Linha de Produção'
               }
vars = ['P-PDG',
        'P-TPT',
        'T-TPT',
        'P-MON-CKP',
        'T-JUS-CKP',
        'P-JUS-CKGL',
        'T-JUS-CKGL',
        'QGL']
columns = ['timestamp'] + vars + ['class'] 
normal_class_code = 0
abnormal_classes_codes = [1, 2, 5, 6, 7, 8]
sample_size = 3*60              # Nas observações = segundos
min_normal_period_size = 20*60  # Nas observações = segundos
split_range = 0.6               # Porcentagem de separação entre treino/teste
max_samples_per_period = 15     # limitação por 'segurança'
df_fc_p = MinimalFCParameters() # Ver documentação da biblioteca tsfresh - opção: EfficientFCParameters()
df_fc_p.pop('sum_values')       # Remove feature inapropriada
df_fc_p.pop('length')           # Remove feature inapropriada
max_nan_percent = 0.1           # Para seleção de variáveis
std_vars_min = 0.01             # Para seleção de variáveis
clfs = {}                       # Dicionário para lista de classificadores a serem experimentados
disable_progressbar = True      # Para menos saídas no notebook

In [7]:
def class_and_file_generator(data_path, real=False, simulated=False, drawn=False):
    """Gerador de lista contendo número da classe e caminho do arquivo de acordo com a fonte da instância."""    
    for class_path in data_path.iterdir():
        if class_path.is_dir():
            class_code = int(class_path.stem)
            for instance_path in class_path.iterdir():
                if (instance_path.suffix == '.csv'):
                    if (simulated and instance_path.stem.startswith('SIMULATED')) or \
                       (drawn and instance_path.stem.startswith('DRAWN')) or \
                       (real and (not instance_path.stem.startswith('SIMULATED')) and \
                       (not instance_path.stem.startswith('DRAWN'))):
                        yield class_code, instance_path

In [8]:
def load_instance(instance_path):
    """Função que carrega cada instância individualmente"""
    try:
        well, instance_id = instance_path.stem.split('_')
        df = pd.read_csv(instance_path, sep=',', header=0)
        assert (df.columns == columns).all(), \
            f'Colunas inválidas no arquivo {str(instance_path)}: {str(df.columns.tolist())}'
        return df
    except Exception as e:
        raise Exception(f'Erro ao ler arquivo {instance_path}: {e}')

In [9]:
def extract_samples(df, class_code):
    # Obtém os rótulos das observações e seu conjunto inequívoco
    ols = list(df['class'])
    set_ols = set()
    for ol in ols:
        if ol in set_ols or np.isnan(ol):
            continue
        set_ols.add(int(ol))       
    
    # Descarta os rótulos das observações e substitui todos os nan por 0
    # (requisito da biblioteca tsfresh)
    df_vars = df.drop('class', axis=1).fillna(0)  
    
    # Inicializa objetos que serão retornados
    df_samples_train = pd.DataFrame()
    df_samples_test = pd.DataFrame()
    y_train = []
    y_test = []
            
    # Descubre o número máximo de amostras em períodos normais, transitórios e em regime
    # Obtém índices (primeiro e último) sem sobreposição com outros períodos
    f_idx = ols.index(normal_class_code)
    l_idx = len(ols)-1-ols[::-1].index(normal_class_code)

    # Define o número inicial de amostras para o período normal
    max_samples_normal = l_idx-f_idx+1-sample_size
    if (max_samples_normal) > 0:      
        num_normal_samples = min(max_samples_per_period, max_samples_normal)
        num_train_samples = int(split_range*num_normal_samples)
        num_test_samples = num_normal_samples - num_train_samples    
    else:
        num_train_samples = 0
        num_test_samples = 0
    
    # Define o número máximo de amostras por período transitório
    transient_code = class_code + 100    
    if transient_code in set_ols:
        # Obtém índices (primeiro e último) com possível sobreposição
        # no início do período
        f_idx = ols.index(transient_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(transient_code)        
        max_transient_samples = l_idx-f_idx+1-sample_size
    else:
        max_transient_samples = 0            

    # Define o número máximo de amostras no período de regime
    if class_code in set_ols:
        # Obtém índices (primeiro e último) com possível sobreposição 
        # no início ou fim do período
        f_idx = ols.index(class_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(class_code)
        if l_idx+(sample_size-1) < len(ols)-1:
            l_idx = l_idx+(sample_size-1) 
        else:
            l_idx = len(ols)-1
        max_in_regime_samples = l_idx-f_idx+1-sample_size
    else:
        max_in_regime_samples = 0   
        
    # Descubre o número adequado de amostras em períodos normais, transitórios e em regime
    num_transient_samples = ceil(num_test_samples/2)
    num_in_regime_samples = num_test_samples - num_transient_samples
    if (max_transient_samples >= num_transient_samples) and \
       (max_in_regime_samples < num_in_regime_samples):
        num_in_regime_samples = max_in_regime_samples        
        num_transient_samples = min(num_test_samples-num_in_regime_samples, max_transient_samples)
    elif (max_transient_samples < num_transient_samples) and \
         (max_in_regime_samples >= num_in_regime_samples):
        num_transient_samples = max_transient_samples        
        num_in_regime_samples = min(num_test_samples-num_transient_samples, max_in_regime_samples)
    elif (max_transient_samples < num_transient_samples) and \
         (max_in_regime_samples < num_in_regime_samples):
        num_transient_samples = max_transient_samples
        num_in_regime_samples = max_in_regime_samples
        num_test_samples = num_transient_samples+num_in_regime_samples
    
    # Extrai amostras do período normal para treinamento e teste
    # Obtém índices (primeiro e último) sem sobreposição com outros períodos
    f_idx = ols.index(normal_class_code)
    l_idx = len(ols)-1-ols[::-1].index(normal_class_code)
    
    # Define a etapa correta e extrai amostras
    if (num_normal_samples) > 0:  
        if num_normal_samples == max_samples_normal:
            step_max = 1 
        else:
            step_max = (max_samples_normal-1) // (max_samples_per_period-1)
        step_wanted = sample_size
        step = min(step_wanted, step_max)
        
        # Extrai amostras para treinamento
        sample_id = 0
        for idx in range(num_train_samples):
            f_idx_c = l_idx-sample_size+1-(num_normal_samples-1-idx)*step
            l_idx_c = f_idx_c+sample_size
            df_sample = df_vars.iloc[f_idx_c:l_idx_c, :]
            df_sample.insert(loc=0, column='id', value=sample_id)
            df_samples_train = df_samples_train.append(df_sample)
            y_train.append(normal_class_code)
            sample_id += 1
    
        # Extrai amostras para teste
        sample_id = 0
        for idx in range(num_train_samples, num_train_samples+num_test_samples):
            f_idx_c = l_idx-sample_size+1-(num_normal_samples-1-idx)*step
            l_idx_c = f_idx_c+sample_size
            df_sample = df_vars.iloc[f_idx_c:l_idx_c, :]
            df_sample.insert(loc=0, column='id', value=sample_id)
            df_samples_test = df_samples_test.append(df_sample)
            y_test.append(normal_class_code)
            sample_id += 1

    # Extrai amostras do período transitório (se existir) para teste
    if (num_transient_samples) > 0:    
        # Define a etapa correta e extrai amostras
        if num_transient_samples == max_transient_samples:
            step_max = 1 
        else:
            step_max = (max_transient_samples-1) // (max_samples_per_period-1)
        step_wanted = np.inf
        step = min(step_wanted, step_max)
        
        # Obtém índices (primeiro e último) com possível sobreposição no início deste período
        f_idx = ols.index(transient_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(transient_code) 

        # Extrai amostras
        for idx in range(num_transient_samples):
            f_idx_c = f_idx+idx*step
            l_idx_c = f_idx_c+sample_size
            df_sample = df_vars.iloc[f_idx_c:l_idx_c, :]
            df_sample.insert(loc=0, column='id', value=sample_id)
            df_samples_test = df_samples_test.append(df_sample)
            y_test.append(transient_code)
            sample_id += 1
            
    # Extrai amostras do período em regime (se existir) para teste
    if (num_in_regime_samples) > 0:     
        # Define a etapa correta e extrai amostras
        if num_in_regime_samples == max_in_regime_samples:
            step_max = 1 
        else:
            step_max = (max_in_regime_samples-1) // (max_samples_per_period-1)
        step_wanted = sample_size
        step = min(step_wanted, step_max)
        
        # Obtém índices (primeiro e último) com possível sobreposição 
        # no início ou no final deste período
        f_idx = ols.index(class_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(class_code)
        if l_idx+(sample_size-1) < len(ols)-1:
            l_idx = l_idx+(sample_size-1) 
        else:
            l_idx = len(ols)-1

        # Extrai amostras
        for idx in range(num_in_regime_samples):
            f_idx_c = f_idx+idx*step
            l_idx_c = f_idx_c+sample_size
            df_sample = df_vars.iloc[f_idx_c:l_idx_c, :]
            df_sample.insert(loc=0, column='id', value=sample_id)
            df_samples_test = df_samples_test.append(df_sample)
            y_test.append(class_code)
            sample_id += 1
            
    #print(f'df_samples_train (antes de normalizar):')
    #display(df_samples_train)
    #print(f'y_train (antes de ajustar para +1 e -1): {y_train} \n')
    #print(f'df_samples_test (antes de normalizar):')
    #display(df_samples_test)
    #print(f'y_test (antes de ajustar para +1 e -1): {y_test} \n')
    
    return df_samples_train, y_train, df_samples_test, y_test              

In [10]:
def train_test_calc_scores(X_train, y_train, X_test, y_test, scores, clfs):
    X_train.reset_index(inplace=True, drop=True)
    X_test.reset_index(inplace=True, drop=True)    
    for clf_name, clf in clfs.items():
        #print(f'CLASSIFICADOR: {clf_name}')
        #print(f'y_train: {y_train}')
        #print(f'y_test: {y_test}')
        
        # Treino
        t0 = time()
        clf.fit(X_train, y_train)
        t_train = time() - t0

        # Teste
        t0 = time()
        y_pred = clf.predict(X_test)
        #print(f'y_pred: {y_pred}')
        t_test = time() - t0

        # Plota os labels reais e preditos pelo classificador
        """
        fig = plt.figure(figsize=(12,1))
        ax = fig.add_subplot(111)
        plt.plot(-(y_pred), marker=11, color='orange', linestyle='') 
        plt.plot(-(y_test), marker=10, color='green', linestyle='')  
        ax.grid(False)
        ax.set_yticks([-1, 1])
        ax.set_yticklabels(['Normal', 'Anormal'])
        ax.set_title(clf_name)            
        ax.set_xlabel('Amostra')
        ax.legend(['Classe Prevista', 'Classe Real'])
        plt.show()
        """
        
        # Calcula as metricas de desempenho
        ret = precision_recall_fscore_support(y_test, y_pred, average='micro')
        p, r, f1, _ = ret
        scores = scores.append({'CLASSIFICADOR': clf_name, 
                                'PRECISAO': p,
                                'REVOGACAO': r,
                                'F1': f1,
                                'TREINAMENTO [s]': t_train, 
                                'TESTE [s] ': t_test}, ignore_index=True)  
    return scores

In [11]:
# Gets all real instances but maintains only those with any type of undesirable event
real_instances = pd.DataFrame(class_and_file_generator(data_path, 
                                                       real=True,
                                                       simulated=False, 
                                                       drawn=False),
                              columns=['class_code', 'instance_path'])
real_instances = real_instances.loc[real_instances.iloc[:,0].isin(abnormal_classes_codes)].reset_index(drop=True)

In [12]:
# For each real instance with any type of undesirable event
scores = pd.DataFrame()
ignored_instances = 0
used_instances = 0
for i, row in real_instances.iterrows():
    # Loads the current instance
    class_code, instance_path = row
    print(f'____________________________________________________________________________________')        
    print(f'Instância {i+1}: {instance_path}')
    df = load_instance(instance_path)
    
    # Ignores instances without sufficient normal periods
    normal_period_size = (df['class']==float(normal_class_code)).sum()
    if normal_period_size < min_normal_period_size:
        ignored_instances += 1
        print(f'\tignorado porque normal_period_size é insuficiente para treinamento ({normal_period_size})\n')
        continue
    used_instances += 1
        
    # Extracts samples from the current real instance
    df_samples_train, y_train, df_samples_test, y_test = extract_samples(df, class_code)

    # Changes types of the labels (tsfresh's requirement)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    # (Wander) incluidas as duas linhas abaixo para ajustar y_train também (?)
    y_train[y_train!=normal_class_code] = -1
    y_train[y_train==normal_class_code] = 1

    # We want binary classification: 1 for inliers (negative class = normal instance) and
    # -1 for outliers (positive class = instance with anomaly) (sklearn's requirement)
    y_test[y_test!=normal_class_code] = -1
    y_test[y_test==normal_class_code] = 1
    
    # Drops the bad vars
    good_vars = np.isnan(df_samples_train[vars]).mean(0) <= max_nan_percent
    std_vars = np.nanstd(df_samples_train[vars], 0)
    good_vars &= (std_vars > std_vars_min)    
    good_vars = list(good_vars.index[good_vars])
    bad_vars = list(set(vars)-set(good_vars))
    df_samples_train.drop(columns=bad_vars, inplace=True, errors='ignore')
    df_samples_test.drop(columns=bad_vars, inplace=True, errors='ignore')
    
    # Normalizes the samples (zero mean and unit variance)
    scaler = preprocessing.StandardScaler()
    df_samples_train[good_vars] = scaler.fit_transform(df_samples_train[good_vars]).astype('float32')
    df_samples_test[good_vars] = scaler.transform(df_samples_test[good_vars]).astype('float32')
    
    # Extracts features from samples
    X_train = extract_features(df_samples_train, 
                               column_id='id', 
                               column_sort='timestamp', 
                               default_fc_parameters=df_fc_p,
                               impute_function=impute,
                               n_jobs=0,
                               disable_progressbar=disable_progressbar)
    X_train = X_train.reset_index(drop=True)
    X_test = extract_features(df_samples_test, 
                              column_id='id', 
                              column_sort='timestamp',
                              default_fc_parameters=df_fc_p,
                              impute_function=impute,
                              n_jobs=0,
                              disable_progressbar=disable_progressbar)
    X_test = X_test.reset_index(drop=True)
    
    #print(f'df_samples_train (normalizado e tratado):')
    #display(df_samples_train)
    #print(f'df_samples_test (normalizado):')
    #display(df_samples_test)
    #print(f'X_train:')
    #display(X_train)
    #print(f'y_train (ajustado para +1 e -1): {y_train} \n')
    #print(f'X_test:')
    #display(X_test)
    #print(f'y_test (ajustado para +1 e -1): {y_test} \n')

    # LISTA DE CLASSIFICADORES A SEREM EXPERIMENTADOS
        
    # DUMMY - Classificador ingênuo
    clfs['Dummy'] = DummyClassifier(strategy='constant', constant=1)
    
    # ISOLATION FOREST - busca de melhores hiperparâmetros (384 combinações)
    isolation_forest_params = {
        'n_estimators': [50, 100, 150, 200],
        'max_samples': ['auto', 0.50, 0.75, 1.0],
        'contamination': ['auto', 0, 0.05, 0.10],
        'bootstrap': [True, False],
        'max_features': [0.50, 0.75, 1.0],
        'random_state': [random_state]
    }
    for params in ParameterGrid(isolation_forest_params):
        isolation_forest_clf = IsolationForest().set_params(**params)
        clfs[f'Floresta de Isolamento: {params}'] = isolation_forest_clf

    # ONE CLASS SVM - busca de melhores hiperparâmetros (240 combinações)
    ocsvm_params = {
        'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
        'gamma': ['auto', 'scale', 1e-4, 1e-3, 1e-2, 0.1, 0.50, 1.0, 5.0, 10.0],
        'nu': [1e-4, 1e-3, 1e-2, 0.10, 0.50, 1.0]
    }    
    for params in ParameterGrid(ocsvm_params):
        ocsvm_clf = OneClassSVM().set_params(**params)
        clfs[f'One-Class SVM: {params}'] = ocsvm_clf
    
    # LOCAL OUTLIER FACTOR (LOF) - busca de melhores hiperparâmetros (1056 combinações)
    lof_params = {
        'n_neighbors': [5, 10, 15, 20],
        'algorithm': ['auto'],
        'leaf_size': [15, 30, 45], # Leaf size passed to BallTree or KDTree algorithm. 
        'metric': ['cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan', 'braycurtis', 
                   'canberra', 'chebyshev', 'correlation', 'dice', 'hamming', 'jaccard', 
                   'kulsinski', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 
                   'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule'],
        'contamination': ['auto', 0.01, 0.05, 0.10],
        'novelty': [True]
    }    
    for params in ParameterGrid(lof_params):
        lof_clf = LocalOutlierFactor().set_params(**params)
        clfs[f'Local Outlier Factor (LOF): {params}'] = lof_clf
    
    # ELLIPTIC ENVELOPE(LOF) - busca de melhores hiperparâmetros (36 combinações)
    ellipticenvelope_params = {
        'contamination': [1e-4, 1e-3, 0.01, 0.05, 0.10, 0.50],
        'assume_centered': [True, False],
        'support_fraction': [0.80, 0.90, 0.99], # proportion of points to be included in the support of the raw MCD estimate. 
        'random_state': [random_state]
    }    
    for params in ParameterGrid(ellipticenvelope_params):
        ellipticenvelope_clf = EllipticEnvelope().set_params(**params)
        clfs[f'Envelope Elíptico: {params}'] = ellipticenvelope_clf
    

    # Preenchido abaixo somente apos encontrar os melhores parametros
    clfs_only_best = {'One Class SVM':     OneClassSVM(kernel='rbf', gamma=0.001, nu=0.1),
            'Floresta de Isolamento':        IsolationForest(bootstrap=False, contamination=0, 
                                                             max_features=1.0, max_samples=1.0, 
                                                             n_estimators=150, random_state=1),
            'Envelope Eliptico': EllipticEnvelope(assume_centered=False, contamination=0.0001, 
                                                  random_state=1, support_fraction=0.8),
            'Local Outlier Factor': LocalOutlierFactor(algorithm='auto', contamination='auto', 
                                                       leaf_size=15, metric='chebyshev', 
                                                       n_neighbors=10, novelty=True),
            'AutoEncoder': AutoEncoder(hidden_neurons=[8, 4, 4, 8], hidden_activation='relu', 
                                               output_activation='sigmoid', loss=keras.losses.mean_squared_error, 
                                               optimizer='adam', epochs=20, batch_size=2, dropout_rate=0.2, 
                                               l2_regularizer=0.1, validation_size=0.1, 
                                               preprocessing=True, verbose=0, 
                                               random_state=1, contamination=0.1),
            'Dummy': DummyClassifier(strategy='constant', constant=1)
            } 
    
    # Treina, testa e calcula os scores para cada classificador na instância    
    
    # Para todas as combinações (usando clfs):
    # scores = train_test_calc_scores(X_train, y_train, X_test, y_test, scores, clfs)
    
    # Somente para a melhor combinação de cada um (usando clfs_only_best):
    scores = train_test_calc_scores(X_train, y_train, X_test, y_test, scores, clfs_only_best)

____________________________________________________________________________________
Instância 1: data\1\WELL-00001_20140124213136.csv
	ignorado porque normal_period_size é insuficiente para treinamento (959)

____________________________________________________________________________________
Instância 2: data\1\WELL-00002_20140126200050.csv
	ignorado porque normal_period_size é insuficiente para treinamento (1138)

____________________________________________________________________________________
Instância 3: data\1\WELL-00006_20170801063614.csv
____________________________________________________________________________________
Instância 4: data\1\WELL-00006_20170802123000.csv
____________________________________________________________________________________
Instância 5: data\1\WELL-00006_20180618060245.csv
____________________________________________________________________________________
Instância 6: data\2\WELL-00002_20131104014101.csv
_______________________________________

## 3. Resultados

Os resultados obtidos com os métodos implementados são apresentados abaixo.

In [13]:
print(f'Número de instâncias utilizadas: {used_instances}')
print(f'Número de instâncias ignoradas: {ignored_instances}')

Número de instâncias utilizadas: 36
Número de instâncias ignoradas: 16


In [14]:
print(f'Características utilizadas: {list(df_fc_p.keys())}')

Características utilizadas: ['median', 'mean', 'standard_deviation', 'variance', 'maximum', 'minimum']


Os comandos a seguir permitem salvar e recuperar os resultados de/para um arquivo CSV de forma conveniente.

In [15]:
scores.to_csv(r'./results/anomaly_detection_scores_por_rodada.csv')

### 3.1. Métricas em formato tabular

As tabelas a seguir apresentam as médias e o desvio padrão das métricas, respectivamente. Ambos são ordenados pela medida-F1.

In [16]:
# Médias
mean_score_table = scores.groupby('CLASSIFICADOR').mean().sort_values(by=['F1'], ascending=False)
mean_score_table.to_csv(r'./results/anomaly_detection_scores_medias.csv')
mean_score_table

,F1,PRECISAO,REVOGACAO,TESTE [s],TREINAMENTO [s]
CLASSIFICADOR,,,,,
Local Outlier Factor,0.881944,0.881944,0.881944,0.002051,0.001841
Floresta de Isolamento,0.743056,0.743056,0.743056,0.126486,0.487763
Envelope Eliptico,0.664352,0.664352,0.664352,0.001673,0.051549
One Class SVM,0.567130,0.567130,0.567130,0.001283,0.001718
Dummy,0.500000,0.500000,0.500000,0.000192,0.000273
AutoEncoder,0.439815,0.439815,0.439815,0.008975,4.241497


In [17]:
# Desvios Padrão
std_score_table = scores.groupby('CLASSIFICADOR').std().sort_values(by=['F1'], ascending=True)
std_score_table.to_csv(r'./results/anomaly_detection_scores_desvios_padrao.csv')
std_score_table

,F1,PRECISAO,REVOGACAO,TESTE [s],TREINAMENTO [s]
CLASSIFICADOR,,,,,
Dummy,0.000000,0.000000,0.000000,0.000397,0.000461
Local Outlier Factor,0.126577,0.126577,0.126577,0.000771,0.000640
AutoEncoder,0.137453,0.137453,0.137453,0.002714,0.845468
Envelope Eliptico,0.157468,0.157468,0.157468,0.000592,0.011521
One Class SVM,0.162836,0.162836,0.162836,0.000656,0.000631
Floresta de Isolamento,0.179699,0.179699,0.179699,0.028920,0.098157


### 3.2. Testes estatísticos

Utilizado teste de Friedman e teste de Holm.

In [18]:
# Teste Estatístico (Friedman)

clfs_names = list(clfs_only_best.keys())
f1s = [scores.loc[scores['CLASSIFICADOR']==cn, 'F1'].values for cn in clfs_names]
f_value_stat, p_value, ranks, pivots = stac.friedman_test(*(f1s))
print(f'p_value: {p_value}')

p_value: 1.1102230246251565e-16


In [19]:
# Teste Estatístico (Holm)

comp, z_values_stat, p_values, adj_p_values = stac.holm_test(len(pivots), pivots, clfs_names, clfs_names.index('Dummy'))
for i in range(len(comp)):
    print(f'{comp[i]}: \n\tp_values: {p_values[i]}\n\tadj_p_values: {adj_p_values[i]}')

Dummy vs Local Outlier Factor: 
	p_values: 3.4439118223872356e-13
	adj_p_values: 1.7219559111936178e-12
Dummy vs Floresta de Isolamento: 
	p_values: 4.254053665997404e-06
	adj_p_values: 1.7016214663989615e-05
Dummy vs Envelope Eliptico: 
	p_values: 0.0008417560142277569
	adj_p_values: 0.0025252680426832708
Dummy vs AutoEncoder: 
	p_values: 0.2313504077741384
	adj_p_values: 0.4627008155482768
Dummy vs One Class SVM: 
	p_values: 0.27028938480169806
	adj_p_values: 0.4627008155482768


In [20]:
# Calcular tempo total do notebook Jupyter
print(f'Tempo total de execução (hh:mm:ss.ms): {datetime.now() - start_time}')

Tempo total de execução (hh:mm:ss.ms): 0:03:24.636026
